In [ ]:
%%capture
!pip install openai

In [ ]:
# Import libraries
import pandas as pd
from google.colab import files

import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [ ]:
# Upload file, read the uploaded CSV into a DataFrame
uploaded = files.upload()
file_name = next(iter(uploaded))
df = pd.read_csv(file_name)

print(df)

Saving general_knowledge_questions.csv to general_knowledge_questions (2).csv
                                              Questions
0              Which planet is known as the red planet?
1                            Who painted the Mona Lisa?
2     What is the largest country in the world by la...
3                What is the capital city of Australia?
4                 What is the chemical symbol for gold?
...                                                 ...
6496              97 Who wrote the novel "Animal Farm"?
6497           98 What is the national animal of China?
6498                 99 Who painted "The Starry Night"?
6499  100 What is the process by which water vapor t...
6500  Please note that these questions are not just ...

[6501 rows x 1 columns]


In [3]:
# OpenAI API key
openai.api_key = "your key here"

GPT_MODEL = "gpt-3.5-turbo-0613"

In [5]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model = GPT_MODEL):
    """
    Send a request to the OpenAI API to get chat completions.

    Args:
        messages (list): The messages to be sent for completion.
        tools (list, optional): List of tools to be used in the API call.
        tool_choice (str, optional): Tool choice parameter for the API call.
        model (str, optional): Model identifier for the API request. Defaults to DEFAULT_GPT_MODEL.

    Returns:
        dict: The API response in JSON format or an error message.

    Raises:
        Exception: Reraises any exception that isn't handled by the retry mechanism.
    """

    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if tools is not None:
        json_data.update({"tools": tools})
    if tool_choice is not None:
        json_data.update({"tool_choice": tool_choice})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response.json()
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [ ]:
def ask_question(question):
    """
    Formulate a question for a chat model and retrieve the response.

    Args:
        question (str): The question to be asked to the chat model.

    Returns:
        str: The response from the chat model.
    """
    messages = []
    messages.append({"role": "system", "content": """Answer the question. Your answer
should be direct, clear, short and simple."""})
    messages.append({"role": "assistant", "content": question})
    chat_response = chat_completion_request(messages)
    assistant_message = chat_response["choices"][0]["message"]["content"]
    return assistant_message

In [ ]:
# Example usage
question = "what is the name of justin bieber brother?"
response = ask_question(question)
print(response)

Justin Bieber's brother's name is Jaxon Bieber.


In [ ]:
new_df = pd.DataFrame(columns=['question', 'answer'])

for index, row in df.iterrows():
    question = row['Questions']
    response = ask_question(question)

    # Append the question and response to the new DataFrame
    new_df = pd.concat([new_df, pd.DataFrame({'question': [question], 'answer': [response]})], ignore_index=True)

# Display the new DataFrame
print(new_df)

                                               question  \
0              Which planet is known as the red planet?   
1                            Who painted the Mona Lisa?   
2     What is the largest country in the world by la...   
3                What is the capital city of Australia?   
4                 What is the chemical symbol for gold?   
...                                                 ...   
6496              97 Who wrote the novel "Animal Farm"?   
6497           98 What is the national animal of China?   
6498                 99 Who painted "The Starry Night"?   
6499  100 What is the process by which water vapor t...   
6500  Please note that these questions are not just ...   

                                                 answer  
0                                                 Mars.  
1                                    Leonardo da Vinci.  
2                                               Russia.  
3                                             Canberra.  
4

In [ ]:
new_df.head()

,question,answer
0,Which planet is known as the red planet?,Mars.
1,Who painted the Mona Lisa?,Leonardo da Vinci.
2,What is the largest country in the world by la...,Russia.
3,What is the capital city of Australia?,Canberra.
4,What is the chemical symbol for gold?,The chemical symbol for gold is Au.


In [ ]:
new_df.to_csv('newQA.csv', index=False)

from google.colab import files
files.download('newQA.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def generate_questions():
    """
    Request the chat model to generate a list of 10 general knowledge questions.

    Returns:
        str: The generated list of questions as a string.
    """
    messages = []
    messages.append({"role": "system", "content": """Generate 10 general knowledge questions. This questions
    should be clear and precise, the should not be repeated"""})
    chat_response = chat_completion_request(messages)
    assistant_message = chat_response["choices"][0]["message"]["content"]
    return assistant_message

In [ ]:
# Generated questions
response = generate_questions()
print(response)

1. Which continent is the largest by land area?
2. Who painted the Mona Lisa?
3. What is the capital city of Australia?
4. What is the chemical symbol for gold?
5. Which planet is known as the "Red Planet"?
6. Who wrote the famous novel "To Kill a Mockingbird"?
7. What is the world's longest river?
8. What is the tallest mountain in the world?
9. How many players are there on a basketball team?
10. In which country would you find the Taj Mahal?


In [ ]:
def generate_questions():
    """
    Generate a large number of unique general knowledge questions by making repeated calls to a chat model API.

    This function initiates 65 separate requests to the chat model API, each requesting the generation of 100 general
    knowledge questions. It compiles all the questions into a single list, ensuring no empty questions are included.

    Returns:
        list: A list containing all the generated questions, with any empty lines removed.
    """
    questions = []
    for _ in range(65):
        messages = []
        messages.append({"role": "system", "content": """Generate 100 general knowledge questions."""})
        chat_response = chat_completion_request(messages)
        assistant_message = chat_response["choices"][0]["message"]["content"]
        print("Assistant message:", assistant_message)
        new_questions = assistant_message.split('\n')
        # Remove empty strings
        new_questions = [q for q in new_questions if q.strip()]
        questions.extend(new_questions)
    return questions

questions = generate_questions()

df_gen_quest = pd.DataFrame({"Questions": questions})
print("Done")

Streaming output truncated to the last 5000 lines.
4. What is the capital city of Australia?
5. Which country is known as the "Land of the Rising Sun"?
6. Who wrote the novel "1984"?
7. What is the chemical symbol for gold?
8. In which year did World War II end?
9. How many continents are there in the world?
10. In which country is the Great Barrier Reef located?
11. What is the tallest mountain on Earth?
12. Who painted the ceiling of the Sistine Chapel?
13. Which country is home to the Taj Mahal?
14. What is the largest organ in the human body?
15. Who wrote the plays "Romeo and Juliet" and "Hamlet"?
16. Which country is known for the Great Wall?
17. What is the currency of Japan?
18. Who was the first person to set foot on the moon?
19. Which element is represented by the symbol Fe on the periodic table?
20. Who painted the artwork "The Starry Night"?
21. How many players are there in a basketball team?
22. What is the largest desert on Earth?
23. Which city is known as the "City of

In [ ]:
# Remove numbers from the beginning of each question
df_gen_quest['Questions'] = df_gen_quest['Questions'].str.replace(r'^\d+\.\s*', '', regex=True)

print(df_gen_quest.head(3))

                                           Questions
0           Which planet is known as the red planet?
1                         Who painted the Mona Lisa?
2  What is the largest country in the world by la...


In [ ]:
# Save dataframe to a CSV file
df_gen_quest.to_csv("general_knowledge_questions.csv", index=False)
# Download the CSV file to your local machine
from google.colab import files
files.download('general_knowledge_questions.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>